In [1]:
import xml.etree.ElementTree
import math
import numpy as np
from scipy.spatial.transform import Rotation as R

In [5]:
def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

In [10]:

tree = xml.etree.ElementTree.parse("generated/generatedURDF.urdf")

#Normalize all joint positions
for joint in tree.getroot().findall("./joint"):
    origin = joint.find('./origin')
    old_xyz = [float(i) for i in origin.attrib['xyz'].split(' ')]
    new_xyz = [0.75*i for i in old_xyz]
    origin.attrib['xyz'] = ' '.join([str(i) for i in new_xyz])
#print(xml.etree.ElementTree.tostring(tree.getroot()).decode())
;

#Remove all visual geometries
for link in tree.getroot().findall("./link"):
    if link.find('./visual') is None:
        link.append(xml.etree.ElementTree.fromstring("<visual></visual>"))
    visual = link.find('./visual')
    if visual.find('./origin') is None:
        visual.append(xml.etree.ElementTree.fromstring('<origin xyz="0 0 0"></origin>'))
    if visual.find('./geometry') is None:
        visual.append(xml.etree.ElementTree.fromstring('<geometry></geometry>'))
    geometry = visual.find('./geometry')
    geom_kids = geometry.findall("./")
    for kid in geom_kids:
        geometry.remove(kid)

#Add cylinders that go from the parent to the child
for link in tree.getroot().findall("./link"):
    #Find joint for which the link is the child, and the link for which the link is the parent
    link_name = link.get('name')
    
    parent = tree.getroot().find(f"./joint/parent[@link='{link_name}']/..")
    child = tree.getroot().find(f"./joint/child[@link='{link_name}']/..")
    if child is None:
        print(f"Link {link_name} is not a child of any joint")
        link.find('./visual/geometry').append(xml.etree.ElementTree.fromstring(f'<sphere radius="0.025"></sphere>'))
        continue
    
    #Remove rpy from origin
    link_origin =  link.find('./visual/origin')
    if link_origin is not None and "rpy" in link_origin.attrib:
            link_origin.attrib.pop("rpy")
    
    #Child position from parent
    child_position = child.find('./origin').get('xyz').split()
    child_position = [float(i) for i in child_position  ]
    child_position = np.array(child_position)

    if np.linalg.norm(child_position) == 0:
        print(f"Link {link_name} same position as its parent")
        link.find('./visual/geometry').append(xml.etree.ElementTree.fromstring(f'<sphere radius="0.025"></sphere>'))
        continue

    plane_normal = np.cross(np.array([0,0,1]),child_position)
    dot = np.dot(child_position, np.array([0,0,1]))
    phi = np.arccos(dot/np.linalg.norm(child_position))

    # translation_of_z = np.dot(rotation_matrix(plane_normal, phi), np.array([0,0,1]))
    rotation_vector = phi*(plane_normal/np.linalg.norm(plane_normal)) if np.linalg.norm(plane_normal) != 0 else np.array([0,0,0])
    rotation_eulers = R.from_rotvec(rotation_vector).as_euler("xyz")

    roll = rotation_eulers[0]
    pitch = rotation_eulers[1]
    yaw = rotation_eulers[2]
    
    length = sum([(x)**2 for x in child_position])**0.5
    
    #Add Cylinder
    link.find('./visual/geometry').append(xml.etree.ElementTree.fromstring(f'<cylinder length="{length}" radius="0.01"></cylinder>'))
    
    #Add rpy for visualization
    # pitch = math.atan2(child_position[0], child_position[2])
    # yaw = math.asin(child_position[1]/length if length != 0 else 0)
    #When pitch is set to math.pi/2 cylinders are horizontal, when 0 vertical
    #When yaw is 0: compared to when yaw is math.pi/2
    rpy = f"{roll} {pitch} {yaw}"
    
    link.find('./visual/origin').attrib['rpy'] = rpy

    new_xyz = [(-i/2) for i in child_position]
    link.find('./visual/origin').attrib['xyz'] = ' '.join([str(i) for i in new_xyz])

# #Add Inertial Information
# for link in tree.getroot().findall("./link"):
#     #Get visual/origin element
#     if link.get("name") == "base":

#     visual_origin = link.find('./visual/origin')


Link base is not a child of any joint
Link trunk same position as its parent


In [11]:
rpy

'0.0 0.0 0.0'

In [12]:
tree.write("generated/sim_ready_robot.urdf", encoding="utf-8", xml_declaration=True)